# **Homework 2 Phoneme Classification**

## Download dataset

In [241]:
# !curl -L "https://github.com/xraychen/shiny-robot/releases/download/v1.0/libriphone.zip" -o libriphone.zip


### Manually unzip

### Preparing Data

**Helper functions to pre-process the training data from raw MFCC features of each utterance.**

A phoneme may span several frames and is dependent to past and future frames. \
Hence we concatenate neighboring phonemes for training to achieve higher accuracy. The **concat_feat** function concatenates past and future k frames (total 2k+1 = n frames), and we predict the center frame.

Feel free to modify the data preprocess functions, but **do not drop any frame** (if you modify the functions, remember to check that the number of frames are the same as mentioned in the slides)

In [242]:
import os
import random
import pandas as pd
import torch
from tqdm.auto import tqdm, trange


def load_feat(path):
    feat = torch.load(path)
    return feat


def shift(x, n):
    if n < 0:
        left = x[0].repeat(-n, 1)
        right = x[:n]

    elif n > 0:
        right = x[-1].repeat(n, 1)
        left = x[n:]
    else:
        return x

    return torch.cat((left, right), dim=0)


def concat_feat(x, concat_n):
    assert concat_n % 2 == 1  # n must be odd
    if concat_n < 2:
        return x
    seq_len, feature_dim = x.size(0), x.size(1)
    x = x.repeat(1, concat_n)
    x = x.view(seq_len, concat_n,
               feature_dim).permute(1, 0, 2)  # concat_n, seq_len, feature_dim
    mid = (concat_n // 2)
    for r_idx in range(1, mid + 1):
        x[mid + r_idx, :] = shift(x[mid + r_idx], r_idx)
        x[mid - r_idx, :] = shift(x[mid - r_idx], -r_idx)

    return x.permute(1, 0, 2).view(seq_len, concat_n * feature_dim)


def preprocess_data(split,
                    feat_dir,
                    phone_path,
                    concat_nframes,
                    train_ratio=0.8,
                    train_val_seed=1337):
    class_num = 41  # NOTE: pre-computed, should not need change
    mode = 'train' if (split == 'train' or split == 'val') else 'test'

    label_dict = {}
    if mode != 'test':
        phone_file = open(os.path.join(phone_path,
                                       f'{mode}_labels.txt')).readlines()

        for line in phone_file:
            line = line.strip('\n').split(' ')
            label_dict[line[0]] = [int(p) for p in line[1:]]

    if split == 'train' or split == 'val':
        # split training and validation data
        usage_list = open(os.path.join(phone_path,
                                       'train_split.txt')).readlines()
        random.seed(train_val_seed)
        random.shuffle(usage_list)
        percent = int(len(usage_list) * train_ratio)
        usage_list = usage_list[:percent] if split == 'train' else usage_list[
            percent:]
    elif split == 'test':
        usage_list = open(os.path.join(phone_path,
                                       'test_split.txt')).readlines()
    else:
        raise ValueError(
            'Invalid \'split\' argument for dataset: PhoneDataset!')

    usage_list = [line.strip('\n') for line in usage_list]
    print('[Dataset] - # phone classes: ' + str(class_num) +
          ', number of utterances for ' + split + ': ' + str(len(usage_list)))

    max_len = 3000000
    X = torch.empty(max_len, 39 * concat_nframes)
    if mode != 'test':
        y = torch.empty(max_len, dtype=torch.long)

    idx = 0
    for i, fname in tqdm(enumerate(usage_list)):
        feat = load_feat(os.path.join(feat_dir, mode, f'{fname}.pt'))
        cur_len = len(feat)
        feat = concat_feat(feat, concat_nframes)
        if mode != 'test':
            label = torch.LongTensor(label_dict[fname])

        X[idx:idx + cur_len, :] = feat
        if mode != 'test':
            y[idx:idx + cur_len] = label

        idx += cur_len

    X = X[:idx, :]
    if mode != 'test':
        y = y[:idx]

    print(f'[INFO] {split} set')
    print(X.shape)
    if mode != 'test':
        print(y.shape)
        return X, y
    else:
        return X


## Define Dataset

In [243]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader


class LibriDataset(Dataset):

    def __init__(self, X, y=None):
        self.data = X

        if y is not None:
            self.label = torch.LongTensor(y)
        else:
            self.label = None

    def __getitem__(self, index):
        if self.label is not None:
            return self.data[index], self.label[index]
        else:
            return self.data[index]

    def __len__(self):
        return len(self.data)


## Define Model

In [244]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class BasicBlock(nn.Module):

    def __init__(self,
                 input_dim,
                 output_dim,
                 batch_norm=True,
                 dropout_prob=0.0):
        super(BasicBlock, self).__init__()

        self.block = nn.Sequential(
            nn.Linear(input_dim, output_dim),
            nn.BatchNorm1d(output_dim) if batch_norm else nn.Identity(),
            nn.ReLU(),
            nn.Dropout(dropout_prob) if dropout_prob != 0 else nn.Identity())

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.block(x)
        return x


class Classifier(nn.Module):

    def __init__(self,
                 input_dim,
                 output_dim=41,
                 hidden_layers=1,
                 hidden_dim=256,
                 batch_norm=True,
                 dropout_prob=0.0):
        super(Classifier, self).__init__()

        self.fc = nn.Sequential(
            BasicBlock(input_dim, hidden_dim, batch_norm=batch_norm),
            *[
                BasicBlock(hidden_dim,
                           hidden_dim,
                           batch_norm=batch_norm,
                           dropout_prob=dropout_prob)
                for _ in range(hidden_layers)
            ],
            nn.Linear(hidden_dim, output_dim),
        )

    def forward(self, x):
        x = self.fc(x)
        return x


## Hyper-parameters

In [ ]:
# data prarameters

# the number of frames to concat with, n must be odd (total 2k+1 = n frames)
concat_nframes = 31
# the ratio of data used for training, the rest will be used for validation
train_ratio = 0.8

# training parameters

# random seed
seed = 114514
# batch size
batch_size = 512
# the number of training epoch
num_epoch = 50
# learning rate
learning_rate = 1e-4
# the path where the checkpoint will be saved
model_path = './model.ckpt'
# momentum for SGD
momentum = 0.9
# early stop
early_stop = 5

# model parameters

# the input dim of the model, you should not change the value
input_dim = 39 * concat_nframes
# the number of hidden layers
hidden_layers = 20
# the hidden dim
hidden_dim = 1024
# batch normalization
batch_norm = True
# dropout probability
dropout_prob = 0.35
# weight decay
weight_decay = 0


## Prepare dataset and model

In [246]:
import gc

# preprocess data
train_X, train_y = preprocess_data(split='train',
                                   feat_dir='./libriphone/feat',
                                   phone_path='./libriphone',
                                   concat_nframes=concat_nframes,
                                   train_ratio=train_ratio)
val_X, val_y = preprocess_data(split='val',
                               feat_dir='./libriphone/feat',
                               phone_path='./libriphone',
                               concat_nframes=concat_nframes,
                               train_ratio=train_ratio)

# get dataset
train_set = LibriDataset(train_X, train_y)
val_set = LibriDataset(val_X, val_y)

# remove raw feature to save memory
del train_X, train_y, val_X, val_y
gc.collect()

# get dataloader
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)


[Dataset] - # phone classes: 41, number of utterances for train: 3428


0it [00:00, ?it/s]

[INFO] train set
torch.Size([2116368, 507])
torch.Size([2116368])
[Dataset] - # phone classes: 41, number of utterances for val: 858


0it [00:00, ?it/s]

[INFO] val set
torch.Size([527790, 507])
torch.Size([527790])


In [247]:
if torch.cuda.is_available():
    print("I have GPU!")
    device = 'cuda:0'


I have GPU!


In [248]:
import numpy as np


#fix seed
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True


same_seeds(seed)


### Model, Loss function, Optimizer

In [249]:
model = Classifier(input_dim,
                   hidden_layers=hidden_layers,
                   hidden_dim=hidden_dim,
                   batch_norm=batch_norm,
                   dropout_prob=dropout_prob).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(),
                              lr=learning_rate,
                              weight_decay=weight_decay)


## Training

In [250]:
best_acc = 0.0
not_improving = 0

pbar = trange(num_epoch)
for epoch in pbar:

    # training
    train_acc = 0.0
    train_loss = 0.0

    model.train()
    for batch in train_loader:
        features, labels = batch
        features = features.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(features)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # torch.max -> max_values, max_value_indices
        _, train_pred = torch.max(outputs, 1)
        train_acc += (train_pred.detach() == labels.detach()).sum().item()
        train_loss += loss.item()

    # validation
    val_acc = 0.0
    val_loss = 0.0

    model.eval()
    with torch.no_grad():
        for batch in val_loader:
            features, labels = batch
            features = features.to(device)
            labels = labels.to(device)

            outputs = model(features)
            loss = criterion(outputs, labels)

            _, val_pred = torch.max(outputs, 1)
            val_acc += (val_pred.cpu() == labels.cpu()).sum().item()
            val_loss += loss.item()

    train_acc /= len(train_set)
    train_loss /= len(train_loader)
    val_acc /= len(val_set)
    val_loss /= len(val_loader)

    # save best model on validation
    if val_acc > best_acc:
        not_improving = 0
        best_acc = val_acc
        torch.save(model.state_dict(), model_path)
    else:
        not_improving += 1

    # display
    info_str = f"Best Acc: {best_acc:.4f}  " + f"Last Training Acc: {train_acc:.4f}  " + f"Last Training Loss: {train_loss:.4f}  " + f"Last Validation Acc: {val_acc:.4f}  " + (
        f"Not Improved for {not_improving} epoches" if not_improving else "")
    pbar.set_postfix_str(info_str)

    if not_improving >= early_stop:

        break


  0%|          | 0/50 [00:00<?, ?it/s]

## Testing

### Load testing data and model

In [251]:
# load data
test_X = preprocess_data(split='test',
                         feat_dir='./libriphone/feat',
                         phone_path='./libriphone',
                         concat_nframes=concat_nframes)
test_set = LibriDataset(test_X, None)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)


[Dataset] - # phone classes: 41, number of utterances for test: 1078


0it [00:00, ?it/s]

[INFO] test set
torch.Size([646268, 507])


In [252]:
# load model
model = Classifier(input_dim,
                   hidden_layers=hidden_layers,
                   hidden_dim=hidden_dim).to(device)
model.load_state_dict(torch.load(model_path))


<All keys matched successfully>

### Make prediction

In [253]:
pred = np.array([], dtype=np.int32)

model.eval()
with torch.no_grad():
    for batch in tqdm(test_loader):
        features = batch
        features = features.to(device)

        outputs = model(features)

        _, test_pred = torch.max(outputs, 1)

        pred = np.concatenate((pred, test_pred.cpu().numpy()), axis=0)


  0%|          | 0/1263 [00:00<?, ?it/s]

In [254]:
# save prediction
with open('prediction.csv', 'w') as f:
    f.write('Id,Class\n')
    for i, y in enumerate(tqdm(pred)):
        f.write(f'{i},{y}\n')


  0%|          | 0/646268 [00:00<?, ?it/s]